In [ ]:
import numpy as np
import matplotlib.pyplot as plt





def compute_lddt(prediction_filename: str, 
                 ref_model_filename: str, 
                 output_lddt_filename: str):
  %shell stage/bin/lddt -f -p stereo_chemical_props.txt \
                        $prediction_filename \
                        $ref_model_filename | tee $output_lddt_filename
  global_lddt = 0
  lddt_scores = []
  reading_data = False
  for line in open(output_lddt_filename, 'r'):
    if reading_data: 
      columns = line.split()
      if len(columns) == 0: continue
      lddt_scores.append(float(columns[5]))
    if line[:17] == 'Global LDDT score': global_lddt = float(line[18:25])
    if line[:5] == 'Chain': reading_data = True
  return global_lddt, np.array(lddt_scores)



def display_lddt_comparison(predicted_scores,
                            measured_scores,
                            output_figure_filename: str,
                            dpi: int = 140):
  plt.figure(figsize=(8,5), dpi=dpi)
  plt.plot(predicted_scores, color='#ccc', label='Predicted lDDT')
  plt.plot(measured_scores, label='Measured lDDT')
  plt.xlabel('Residues')
  plt.ylim(0, 1)
  plt.legend()
  plt.savefig(output_figure_filename, bbox_inches='tight')
  plt.show()



def compute_metrics(plddtScores, outputFilename: str = 'metrics.txt'):
  global jobname
  global sequences
  global algorithm
  global dpi
  sequence_length = len(sequences[0])
  with open(f'{jobname}/{outputFilename}', 'wt') as file:
    file.write(f'JOBNAME={jobname}\n')
    file.write(f'SEQUENCE_LENGTH={sequence_length}\n')
    file.write(f'ALGORITHM={algorithm}\n')
  %shell /content/pymol-open-source-build/bin/pymol -qc $prediction_filename \
    $ref_model_filename -r /concent/foldeval/pymol_gdt.py 
  global_lddt, lddt_scores = compute_lddt(prediction_filename, ref_model_filename, 
                                          f'{jobname}/lDDT.txt')
  display_lddt_comparison(plddtScores, lddt_scores, 
                          f'{jobname}/lddt_comparison.png', dpi)
  with open(f'{jobname}/{outputFilename}', 'wt') as file:
    file.write(f'LDDT={global_lddt:.4}\n')
    file.write('LDDT_ATOMS=all\n')
    file.write('LDDT_CUTOFFS=[ 0.5, 1, 2, 4 ]\n')
    file.write('LDDT_BOND_ANGLE_TOLERANCE=12\n')
    file.write('LDDT_BOND_LENGTH_TOLERANCE=12\n')
  